In [1]:
# !pip install jiwer pyctcdecode --upgrade -qq

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# import sys
# sys.path.append("../")

In [4]:
from sk import SK
from pathlib import Path
import jiwer
import pandas as pd
from fastcore.basics import partialler
from IPython.display import Audio
pd.set_option('max_colwidth', 400)

# SK class

In [5]:
asr = SK(model="conformer_small",decoder="v1")

Unigrams not provided and cannot be automatically determined from LM file (only arpa format). Decoding accuracy might be reduced.
No known unigrams provided, decoding results might be a lot worse.


loaded model: /home/ubuntu/.sk/models/conformer_small.onnx ['CPUExecutionProvider']
loaded lm: v1


# Predict single or multiple file

In [6]:
fn = "/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ087-000005.wav" # fn=["/content/test/abc.wav","/content/random_test/abc.wav"]
out = asr.transcribe_file(fn=fn)

Total input path: 1
Total audio found(.wav): 1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.20it/s]


In [7]:
Path(fn).with_suffix(".txt").read_text()

'bilakah orang inggeris meminta maaf atas penindasan mereka terhadap orang iraq'

In [8]:
out

{'texts': ['bilakah orang inggeris meminta maaf atas penindasan mereka terhadap orang era'],
 'filenames': ['/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ087-000005.wav'],
 'entropy': [[0.5173527002334595,
   0.671403706073761,
   1.337220549583435,
   1.9809788465499878,
   1.470432996749878,
   0.22879044711589813,
   1.468695044517517,
   1.2725319862365723,
   1.9320794343948364,
   0.4545842409133911,
   1.2927184104919434]],
 'timestamps': [[('bilakah', 0.01, 0.03),
   ('orang', 0.04, 0.05),
   ('inggeris', 1.0, 1.03),
   ('meminta', 1.04, 2.0),
   ('maaf', 2.01, 2.02),
   ('atas', 2.03, 2.05),
   ('penindasan', 2.05, 3.03),
   ('mereka', 3.04, 4.0),
   ('terhadap', 4.01, 4.04),
   ('orang', 4.05, 4.06),
   ('era', 5.01, 5.02)]],
 'speakers': [['not enabled']],
 'all_logits': []}

In [9]:
asr.decoder.decode_beams

<bound method BeamSearchDecoderCTC.decode_beams of <pyctcdecode.decoder.BeamSearchDecoderCTC object at 0x7ff4247434c0>>

In [10]:
# since the model not trained on word containing iraq,can make the model bias toward iraq or can create lm with iraq in the corpus
asr.decoder.decode_beams = partialler(asr.decoder.decode_beams,hotwords=["iraq"],hotword_weight=1e2)

In [11]:
out = asr.transcribe_file(fn=fn)
out

Total input path: 1
Total audio found(.wav): 1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.12it/s]


{'texts': ['bilakah orang inggeris meminta maaf atas penindasan mereka terhadap orang ira'],
 'filenames': ['/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ087-000005.wav'],
 'entropy': [[0.5173527002334595,
   0.671403706073761,
   1.337220549583435,
   1.9809788465499878,
   1.470432996749878,
   0.22879044711589813,
   1.468695044517517,
   1.2725319862365723,
   1.9320794343948364,
   0.4545842409133911,
   1.2927184104919434]],
 'timestamps': [[('bilakah', 0.01, 0.03),
   ('orang', 0.04, 0.05),
   ('inggeris', 1.0, 1.03),
   ('meminta', 1.04, 2.0),
   ('maaf', 2.01, 2.02),
   ('atas', 2.03, 2.05),
   ('penindasan', 2.05, 3.03),
   ('mereka', 3.04, 4.0),
   ('terhadap', 4.01, 4.04),
   ('orang', 4.05, 4.06),
   ('ira', 5.01, 5.02)]],
 'speakers': [['not enabled']],
 'all_logits': []}

In [12]:
Audio(fn)

In [13]:
asr.decoder.decode_beams = partialler(asr.decoder.decode_beams,hotwords=None,hotword_weight=10.) # reset hotwords

# Predict from single or multiple folder

In [14]:
fn = "/content/test-bahasa/" # or fn = ["/content/test/","/content/random_test/"]
out = asr.transcribe_file(fn=fn)

Total input path: 1
Total audio found(.wav): 780


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 780/780 [02:21<00:00,  5.51it/s]


In [15]:
preds,files,entropy,timestamps = out["texts"],out["filenames"],out["entropy"],out["timestamps"]

In [16]:
data = []
for i,j in zip(preds,files):
    label = j.with_suffix('.txt').read_text()
    char_label = [char for char in label]
    char_i = [char for char in i]
    data.append([j,label,i,jiwer.compute_measures(label,i)['wer'],jiwer.compute_measures(char_label,char_i)['wer']])

In [17]:
df = pd.DataFrame(data)
df.columns = ["path","label","pred","wer","cer"]
df = df.sort_values("wer",ascending=False)
df.head(10)

,path,label,pred,wer,cer
356,/content/test-bahasa/wattpad-audio-wattpad-64.wav,mengingatkan yasmin,mengingatkan yang min,1.000000,0.111111
720,/content/test-bahasa/wattpad-audio-wattpad-637.wav,berbalik kearah libra,berbalik ke ada libera,1.000000,0.157895
722,/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-ampe.wav,sebut perkataan ampe,sebuk pakatan ampa,1.000000,0.277778
137,/content/test-bahasa/wattpad-audio-wattpad-646.wav,who are you,ayu,1.000000,0.666667
503,/content/test-bahasa/wattpad-audio-wattpad-88.wav,udah ga cape kita caw yuk,udagacapir kita cayok,0.833333,0.250000
397,/content/test-bahasa/-home-husein-speech-bahasa-streaming-meter nanometer per laksa.wav,mat meter nanometer per laksa,meter nana meter pelaksa,0.800000,0.200000
85,/content/test-bahasa/-home-husein-speech-bahasa-streaming-iaitu paus odontoceti paus.wav,iaitu paus odontoceti paus,iaitu power orden tercetipawes,0.750000,0.434783
629,/content/test-bahasa/wattpad-audio-wattpad-549.wav,haha gak deh mungkin aku terlalu hiperbola tapi bener kalian harus coba,hahagadik mungkin aku terlalu haipebolatapi benda keliha cuba,0.750000,0.233333
237,/content/test-bahasa/wattpad-audio-wattpad-89.wav,libra dan gisel saling bertatapan kemudian mengangguk tanda mengiyakan ajakan,dibaranangisa saling bertetapan kemudia mengangguk tanda mengiakan ajarkan,0.700000,0.132353
745,/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-amangkurat.wav,sebut perkataan amangkurat,sebut perkataan aman kurat,0.666667,0.041667


In [18]:
df["wer"].mean(),df["cer"].mean()

(0.1305165536402763, 0.03635852850835042)